#  Implmention of the multivariate regression

ordinary least square with matrix inversion.According to the multiple linear regression formula of least square method, the formula of regression coefficient can be obtained：beta = (xt * x) ^ -1 * (xt * y).The first coefficient is the intercept and the second coefficient is the slope.And it contains my calculate of r2 and rmse.

In [1]:
import numpy as np
import pandas as pd


class LeastSquare:
    def __init__(self):
        self.intercept = 0
        self.coefficient = []
        self.r2 = 0
        self.rmse = 0

    def fit(self, x, y):
        # 复制x防止改变参数
        x = x[:]
        # 在x的第一列插入一列常数1
        x.insert(0, "constant", [1 for _ in range(len(x))])
        # 求x转置
        xt = x.T
        # (xt * x)
        head = np.dot(xt, x)
        # (xt * y)
        tail = np.dot(xt, pd.DataFrame(y))
        # beta = (xt * x) ^ -1 * (xt * y)
        beta = np.dot(np.linalg.inv(head), tail)
        # 将DataFrame转成list
        a = beta.T[0]
        # 截距
        self.intercept = a[0]
        # 斜率
        self.coefficient = a[1:]

        # 期望值（向量）
        expectations = np.dot(x, beta)
        # 实际值（向量）
        actual = list(y)
        # 平均值（标量）
        average = np.average(actual)

        # 期望值与实际值的平方差
        squire_errors = 0
        for i in range(len(expectations)):
            squire_errors += np.square(expectations[i][0] - actual[i])

        # 平均值与实际值的平方差
        actual_errors = 0
        for i in range(len(actual)):
            actual_errors += np.square(average - actual[i])

        # 计算R2
        self.r2 = 1 - squire_errors / actual_errors
        # 计算RMSE
        self.rmse = np.sqrt(squire_errors / len(actual))

Sk-learn regressor is as following.I'm going to paste the code and compare it to the linear regression That I did.

In [2]:
    def fit(self, X, y, sample_weight=None):

        n_jobs_ = self.n_jobs
        X, y = self._validate_data(X, y, accept_sparse=['csr', 'csc', 'coo'],
                                   y_numeric=True, multi_output=True)

        if sample_weight is not None:
            sample_weight = _check_sample_weight(sample_weight, X,
                                                 dtype=X.dtype)

        X, y, X_offset, y_offset, X_scale = self._preprocess_data(
            X, y, fit_intercept=self.fit_intercept, normalize=self.normalize,
            copy=self.copy_X, sample_weight=sample_weight,
            return_mean=True)

        if sample_weight is not None:
            # Sample weight can be implemented via a simple rescaling.
            X, y = _rescale_data(X, y, sample_weight)

        if sp.issparse(X):
            X_offset_scale = X_offset / X_scale

            def matvec(b):
                return X.dot(b) - b.dot(X_offset_scale)

            def rmatvec(b):
                return X.T.dot(b) - X_offset_scale * np.sum(b)

            X_centered = sparse.linalg.LinearOperator(shape=X.shape,
                                                      matvec=matvec,
                                                      rmatvec=rmatvec)

            if y.ndim < 2:
                out = sparse_lsqr(X_centered, y)
                self.coef_ = out[0]
                self._residues = out[3]
            else:
                # sparse_lstsq cannot handle y with shape (M, K)
                outs = Parallel(n_jobs=n_jobs_)(
                    delayed(sparse_lsqr)(X_centered, y[:, j].ravel())
                    for j in range(y.shape[1]))
                self.coef_ = np.vstack([out[0] for out in outs])
                self._residues = np.vstack([out[3] for out in outs])
        else:
            self.coef_, self._residues, self.rank_, self.singular_ = \
                linalg.lstsq(X, y)
            self.coef_ = self.coef_.T

        if y.ndim == 1:
            self.coef_ = np.ravel(self.coef_)
        self._set_intercept(X_offset, y_offset, X_scale)
        return self

Test class.Test the least square fitting results and compare with Sklearn. And compare my r2 and rmse with the method of sk-learn.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from project1.regression import LeastSquare

# 数据1
tem16_1 = [3113, 3122, 3131, 3137, 3146, 3149, 3157, 3166, 3172, 3178, 3172, 3151, 3137, 3131, 3128, 3107, 3095, 3081,
           3070, 3049, 3038, 3023, 3011, 2997, 2988, 2985, 2974, 2957, 2942, 2928, 2916, 2901, 2887, 2884, 2879, 2870,
           2856, 2848, 2836, 2828, 2810, 2802, 2793, 2785, 2776, 2768, 2762, 2756, 2762, 2765, 2762, 2771, 2748, 2731,
           2708, 2697, 2674, 2697, 2702, 2725, 2737, 2759, 2771, 2787, 2802, 2819, 2833, 2845, 2856, 2859, 2873, 2879,
           2896, 2898, 2910, 2922, 2933, 2945, 2957, 2968, 2980, 2985, 3000, 3011, 3023, 3038, 3046, 3058, 3070, 3072,
           3084, 3090, 3098, 3107, 3119, 3122, 3131, 3134, 3140, 3143, 3149, 3154, 3151, 3166, 3189, 3178, 3181, 3169,
           3166, 3169, 3154, 3137, 3116, 3107, 3090, 3093, 3087, 3075, 3052, 3038, 3026, 3017, 3008, 3002, 2994, 2991,
           2988, 2977, 2957, 2951, 2936, 2931, 2936, 2957, 2980, 2962, 2948, 2928, 2913, 2898, 2879, 2867, 2865, 2856,
           2853, 2848, 2842, 2833, 2830, 2819, 2813, 2807, 2799, 2796, 2787, 2785, 2779, 2773]
# 数据2
tem16_2 = [4185, 4209, 4233, 4256, 4279, 4303, 4326, 4349, 4372, 4398, 4421, 4414, 4387, 4383, 4360, 4349, 4326, 4303,
           4279, 4264, 4233, 4209, 4185, 4161, 4113, 4088, 4064, 4043, 4019, 4023, 4002, 3978, 3954, 3933, 3912, 3891,
           3870, 3845, 3824, 3802, 3781, 3759, 3715, 3693, 3671, 3648, 3626, 3603, 3580, 3534, 3507, 3496, 3490, 3467,
           3451, 3429, 3419, 3432, 3454, 3464, 3486, 3496, 3518, 3542, 3565, 3588, 3611, 3633, 3656, 3678, 3700, 3722,
           3744, 3766, 3788, 3809, 3831, 3873, 3895, 3919, 3940, 3964, 3988, 4010, 4039, 4060, 4084, 4108, 4133, 4157,
           4181, 4205, 4228, 4252, 4276, 4299, 4322, 4345, 4368, 4391, 4414, 4436, 4458, 4455, 4470, 4485, 4462, 4458,
           4451, 4447, 4425, 4402, 4387, 4364, 4368, 4349, 4326, 4307, 4287, 4264, 4240, 4221, 4225, 4197, 4177, 4169,
           4149, 4133, 4113, 4088, 4064, 4043, 4019, 3995, 3985, 3968, 3947, 3926, 3905, 3884, 3863, 3842, 3820, 3799,
           3777, 3773, 3759, 3737, 3715, 3693, 3671, 3648, 3626, 3603, 3576, 3553, 3530, 3507]
# 回归数据
result = [2364, 2356, 2356, 2353, 2353, 2353, 2353, 2367, 2367, 2367, 2383, 2362, 2435, 2460, 2427, 2427, 2451, 2446,
          2424, 2430, 2407, 2399, 2386, 2367, 2372, 2375, 2364, 2342, 2326, 2309, 2285, 2283, 2288, 2277, 2285, 2307,
          2329, 2351, 2372, 2396, 2421, 2492, 2312, 2283, 2280, 2266, 2253, 2239, 2234, 2231, 2247, 2247, 2242, 2220,
          2198, 2098, 2034, 2031, 2053, 2031, 2093, 2071, 2093, 2077, 2142, 2163, 2185, 2206, 2215, 2215, 2215, 2215,
          2217, 2217, 2217, 2217, 2217, 2239, 2239, 2239, 2256, 2256, 2256, 2256, 2274, 2274, 2274, 2274, 2293, 2309,
          2293, 2309, 2331, 2331, 2331, 2331, 2331, 2331, 2331, 2331, 2331, 2331, 2356, 2367, 2380, 2383, 2386, 2370,
          2378, 2372, 2351, 2337, 2320, 2299, 2293, 2312, 2334, 2329, 2323, 2315, 2304, 2307, 2301, 2304, 2290, 2272,
          2256, 2234, 2237, 2245, 2266, 2258, 2261, 2253, 2266, 2245, 2223, 2212, 2215, 2204, 2198, 2201, 2223, 2245,
          2261, 2277, 2269, 2288, 2290, 2315, 2261, 2253, 2266, 2245, 2223, 2212, 2215, 2215]
# 以DataFrame存储数据
data = pd.DataFrame([tem16_1, tem16_2, result], index=['tem16_1', 'tem16_2', 'result'])
# 矩阵转置
data = data.T
# 所有x的二维表
x = data[['tem16_1', 'tem16_2']]
# y的一维表
y = data['result']

# Sk-learn regression训练
linreg = LinearRegression()
linreg.fit(x, y)
# Sk-learn regression结果
print("Sk-learn regression intercept: ", linreg.intercept_)  # 截距
print("Sk-learn regression coefficient: ", linreg.coef_)  # 各个斜率
print("Sk-learn regression r2: ", r2_score(y, linreg.predict(x)))  # r2
print("Sk-learn regression rmse ", np.sqrt(mean_squared_error(y, linreg.predict(x))))  # rmse

# 最小二乘法训练
linreg = LeastSquare()
linreg.fit(x, y)
# 最小二乘法结果
print("linear regression intercept: ", linreg.intercept)  # 截距
print("linear regression coefficient: ", linreg.coefficient)  # 各个斜率
print("linear regression r2: ", linreg.r2)  # r2
print("linear regression rmse: ", linreg.rmse)  # rmse

Sk-learn regression intercept:  1857.4219451566676
Sk-learn regression coefficient:  [-0.33177581  0.35326033]
Sk-learn regression r2:  0.5709359018967981
Sk-learn regression rmse  56.45071056721415
linear regression intercept:  1857.4219451528043
linear regression coefficient:  [-0.33177581  0.35326033]
linear regression r2:  0.5709359018967984
linear regression rmse:  56.450710567214145
